In [6]:
import os
import zipfile
import re
import csv
import shutil

from pathlib import Path

In [7]:
def get_name_and_type(patch):
    match = re.search(r'([A-Za-z\s]+)\s(\d)\b', patch)
    if match:
        name = match.group(1).strip()
        type_ = match.group(2)
        return name, type_
    return None, None

In [8]:
raw_dir   = Path("../patches/raw")
temp_dir  = Path("../patches/temp")
clean_dir = Path("../patches/clean")

clean_dir.mkdir(parents=True, exist_ok=True)

In [9]:
for raw_file in raw_dir.glob("*.mfprojz"):

    patch_temp_dir = temp_dir / raw_file.stem
    patch_temp_dir.mkdir(parents=True, exist_ok=True)
    with zipfile.ZipFile(raw_file, 'r') as zip_ref:
        zip_ref.extractall(patch_temp_dir)

    mbp_files = list(patch_temp_dir.rglob("*.mbp"))
    rows = []
    for mbp_file in mbp_files:
        with open(mbp_file, "r") as f:
            content = f.read()
            name, type_ = get_name_and_type(content)
            if name and type_:
                rel_path = str(mbp_file.relative_to(temp_dir))
                rows.append([name, type_, rel_path])

    csv_path = clean_dir / (raw_file.stem + ".csv")
    with open(csv_path, "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["patch_name", "patch_type", "path"])
        writer.writerows(rows)